In [ ]:
%pip install cohere tiktoken

In [ ]:
# very imp and useful util
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY")

from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# useful util
import sys
import pkg_resources
print("Python interpreter:", sys.executable)
print("Packages:")
for dist in pkg_resources.working_set:
    print(dist)


In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        # this is helps to parse the html given by the url to text
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)


In [ ]:
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
# chunk_siz refers to words, chunk_overlap refers to last 100 words previous chunk and
# first 100 words of next chunk
# startindex adds indexes to the metadata i guess is helpfull when storing in db

In [ ]:
splits = text_splitter.split_documents(docs)
# you can see all indexes that need to be made embeddings to store in vector db maybe, but not sure

In [ ]:

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
# retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
# retrieved_docs[0].page_content

In [ ]:
# prompt with chat history memory
from langchain_core.prompts import PromptTemplate
template = """You are a chatbot having a conversation with a human your are a {ability}.

Given the following extracted parts of a long document and a question,
create a final answer using your intelligence.

(This context can be completely irrelevant to the question if so, skip it,
focus on giving the best answer to the human question)
{context}

{chat_history}
Human: {human_input}
Chatbot:"""

with_history_prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    max_token_limit=100,llm=llm
)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(
    llm, chain_type="stuff", memory=memory, prompt=with_history_prompt, verbose=True
)

In [ ]:
question = "Can you summarize what we did uptil now"

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

docs = retriever.invoke(question)

In [ ]:
response = chain({"input_documents":docs, "human_input": question, "ability": "Good talker"},return_only_outputs=True)

In [ ]:
print(chain.memory.load_memory_variables("chat_history"))
response['output_text']